# MULTI-TASK LEARNING 
In this notebook, I'm going to use all the information seen before to build and end-to-end jointly supervised learning of Depresion and Anxiety.
I will use a new dataset which will consider an encoder (the PHQ-8 pre-trained model) and 2 workers, one for anxiety and one for depresion. As we want all the hard task coming from the encoding and found similarities of co-morbidities, the 2 workers will be just a fully-connected layer from the Q-RNN output of the model to the number of classes of each co-morbidity. 

In [1]:
from sklearn.model_selection import KFold
from data_Loader import dataLoaderPickle, justDatasetLoaderPickle, dataMLTCreator
from networks_v2 import HydraNet, HydraNetTL
from optimizer import optimizerNet
from optimizer import IterMeter

import torch
import torch.nn as nn
from comet_ml import Experiment
import tqdm

import numpy as np
import os 

from torch.utils.data import DataLoader, random_split, SubsetRandomSampler

import torch.nn.functional as F 
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sn 
import pandas as pd 
import matplotlib.pyplot as plt

c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading data
We load the two datasets we have, and combine the targets of each.

In [2]:
train_loader = "C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\Raw_Data_v2\\RADAR-MDD-KCL-s1\\RADAR-MDD-KCL-s1"
test_loader = ""

name =[]
data = []
audio = []

name = [x[0] for x in os.walk(train_loader)]

name.pop(0)

'C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\Raw_Data_v2\\RADAR-MDD-KCL-s1\\RADAR-MDD-KCL-s1'

In [3]:
dataset = dataMLTCreator(train_loader, "MLT_Scripted_Eng")

Signal processed 1_1
New Spectrogram
The new spectrogram has a target
(2, 500, 40) Spectrogram shape
New Spectrogram augmented
The new spectrogram has a target
(2, 500, 40) Spectrogram shape NOISE
DATA AUGMENTED
Length Spectrograms:  (5, 500, 40)
Length labels depresion:  4
Length labels anxiety:  4
Signal processed 2_1
New Spectrogram
The new spectrogram has a target
(2, 500, 40) Spectrogram shape
New Spectrogram augmented
The new spectrogram has a target
(2, 500, 40) Spectrogram shape NOISE
DATA AUGMENTED
Length Spectrograms:  (9, 500, 40)
Length labels depresion:  8
Length labels anxiety:  8
Signal processed 3_1
New Spectrogram
The new spectrogram has a target
(2, 500, 40) Spectrogram shape
Length Spectrograms:  (11, 500, 40)
Length labels depresion:  10
Length labels anxiety:  10
Signal processed 4_1
New Spectrogram
The new spectrogram has a target
(3, 500, 40) Spectrogram shape
Length Spectrograms:  (14, 500, 40)
Length labels depresion:  13
Length labels anxiety:  13
Signal proce

## Building the model
We will use Transfer Learning from the PHQ-8 model. Then charge this model into the HydraNet model.

In [2]:
dataset = justDatasetLoaderPickle('MLT_Scripted_Eng')

In [3]:
#Constants
learning_Rate = 0.0005
batch_size = 128
epochs = 200
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}
experiment = Experiment(api_key='dummy_key', disabled=True)

hparams = {
    "n_cnn_layers": 6,
    "n_rnn_layers": 1,
    "rnn_dim": 256,
    "h_rnn_layers": 128,
    "n_class": 5,
    "n_feats": 64,
    "stride": 2,
    "dropout": 0.3,
    "learning_rate": learning_Rate,
    "batch_size": batch_size,
    "epochs": epochs
}

experiment.log_parameters(hparams)
use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


In [5]:
filename = "C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\model_PHQ8_Scripted_Eng.pt"
infile = open(filename, 'rb')
model_pretrained = torch.load(infile)
infile.close()

In [4]:
#model_mlt = HydraNet(model_pretrained)

#Optimizing Model
#optimizer, scheduler = optimizerNet(model_mlt, hparams)
#criterion = nn.NLLLoss().to(device)
weights_dep = [0.95, 0.85, 1.0, 0.85, 1.0]
class_weights_dep = torch.FloatTensor(weights_dep)
criterion_dep = nn.CrossEntropyLoss(weight=class_weights_dep).to(device)

weights_anx = [1.0,1.0,1.0,0.95]
class_weights_anx = torch.FloatTensor(weights_anx)
criterion_anx = nn.CrossEntropyLoss(weight=class_weights_anx).to(device)
iter_meter = IterMeter()

In [5]:
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    print('Fold {}'.format(fold+1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

    model_mlt = HydraNet(
        hparams['n_cnn_layers'],
        hparams['rnn_dim'],
        hparams['h_rnn_layers'],
        hparams['n_rnn_layers'],
        hparams['n_class'],
        hparams['stride'],
        hparams['dropout']
    ).to(device).float()
    optimizer, scheduler = optimizerNet(model_mlt, hparams)

    acc_dep_list = []
    acc_anx_list = []
    loss_list = []
    label_dep_list = []
    label_anx_list = []
    predicted_list_anx = []
    predicted_list_dep = []
    loss_tot_list = []
    accuracy_dep = []
    accuracy_anx = []

    #Training
    for epoch in range(epochs):
        model_mlt.train()
        total_training_loss = 0

        for x,y,z in train_loader:
            inputs = x
            label_dep = y.long()
            label_anx = z.long()
            label_dep_list.extend(label_dep.detach().numpy())
            label_anx_list.extend(label_anx.detach().numpy())

            optimizer.zero_grad()
            phq8, gad7 = model_mlt(inputs.float())
            phq8 = phq8.squeeze(0)
            gad7 = gad7.squeeze(0)

            predicted_list_dep.extend((torch.max(torch.exp(F.log_softmax(phq8,dim=1)),1)[1]).detach().numpy())
            predicted_list_anx.extend((torch.max(torch.exp(F.log_softmax(gad7,dim=1)),1)[1]).detach().numpy())

            loss_dep = criterion_dep(phq8,label_dep)
            loss_anx = criterion_anx(gad7, label_anx)

            loss = loss_dep + loss_anx
            loss.backward()
            optimizer.step()
            iter_meter.step()
            total_training_loss += loss

            loss_list.append(loss.item())
            #Track accuracy
            total_dep = label_dep.size(0)
            _, predicted = torch.max(phq8.data,1)
            correct_dep = (predicted == label_dep).sum().item()
            acc_dep_list.append(correct_dep/total_dep)  

            total_anx = label_anx.size(0)
            _, predicted = torch.max(gad7.data,1)
            correct_anx = (predicted==label_anx).sum().item()
            acc_anx_list.append(correct_anx/total_anx)

        print('Train Epoch: {} \tLoss: {:.4f}\tDepression Accuracy: {:.4f}\tAnxiety Accuracy: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.mean(acc_dep_list),
            np.mean(acc_anx_list)
        ))      
        loss_tot_list.append(np.mean(loss_list))
        accuracy_dep.append(np.mean(acc_dep_list))
        accuracy_anx.append(np.mean(acc_anx_list))
        
    #Printing Confusion Matrix
    classes_dep = ('0','1','2','3','4')
    cf_matrix_dep = confusion_matrix(label_dep_list, predicted_list_dep)
    df_cm = pd.DataFrame(cf_matrix_dep/np.sum(cf_matrix_dep)*10,index = [i for i in classes_dep], columns=[i for i in classes_dep])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Depression Training')

    classes_anx = ('0','1','2','3')
    cf_matrix_anx = confusion_matrix(label_anx_list, predicted_list_anx)
    df_cm = pd.DataFrame(cf_matrix_anx/np.sum(cf_matrix_anx)*10,index = [i for i in classes_anx], columns=[i for i in classes_anx])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Anxiety Training')

    fig, axs = plt.subplots(3)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Training Loss')
    axs[0].set(xlabel= 'Epoch', ylabel='Loss')
    axs[1].plot(range(epochs), accuracy_dep)
    axs[1].set_title('Training Depression Accuracy')
    axs[1].set(xlabel= 'Epoch', ylabel='Accuracy')
    axs[2].plot(range(epochs), accuracy_anx)
    axs[2].set_title('Training Anxiety Accuracy')
    axs[2].set(xlabel='Epoch', ylabel='Accuracy')
    plt.show()

    #Test
    acc_dep_list = []
    acc_anx_list = []
    loss_list = []
    label_dep_list = []
    label_anx_list = []
    predicted_list_anx = []
    predicted_list_dep = []
    loss_tot_list = []
    accuracy_dep = []
    accuracy_anx = []
    for epoch in range(epochs):
        model_mlt.train()
        total_training_loss = 0

        for x,y,z in test_loader:
            inputs = x
            label_dep = y.long()
            label_anx = z.long()
            label_dep_list.extend(label_dep.detach().numpy())
            label_anx_list.extend(label_anx.detach().numpy())

            optimizer.zero_grad()
            phq8, gad7 = model_mlt(inputs.float())
            phq8 = phq8.squeeze(0)
            gad7 = gad7.squeeze(0)

            predicted_list_dep.extend((torch.max(torch.exp(F.log_softmax(phq8,dim=1)),1)[1]).detach().numpy())
            predicted_list_anx.extend((torch.max(torch.exp(F.log_softmax(gad7,dim=1)),1)[1]).detach().numpy())

            loss_dep = criterion_dep(phq8,label_dep)
            loss_anx = criterion_anx(gad7, label_anx)

            loss = loss_dep + loss_anx
            #loss.backward()
            #optimizer.step()
            #iter_meter.step()
            total_training_loss += loss

            loss_list.append(loss.item())
            #Track accuracy
            total_dep = label_dep.size(0)
            _, predicted = torch.max(phq8.data,1)
            correct_dep = (predicted == label_dep).sum().item()
            acc_dep_list.append(correct_dep/total_dep)  

            total_anx = label_anx.size(0)
            _, predicted = torch.max(gad7.data,1)
            correct_anx = (predicted==label_anx).sum().item()
            acc_anx_list.append(correct_anx/total_anx)

        print('Test Epoch: {} \tLoss: {:.4f}\tDepression Accuracy: {:.4f}\tAnxiety Accuracy: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.mean(acc_dep_list),
            np.mean(acc_anx_list)
        ))      
        loss_tot_list.append(np.mean(loss_list))
        accuracy_dep.append(np.mean(acc_dep_list))
        accuracy_anx.append(np.mean(acc_anx_list))
        
    #Printing Confusion Matrix
    classes_dep = ('0','1','2','3','4')
    cf_matrix_dep = confusion_matrix(label_dep_list, predicted_list_dep)
    df_cm = pd.DataFrame(cf_matrix_dep/np.sum(cf_matrix_dep)*10,index = [i for i in classes_dep], columns=[i for i in classes_dep])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Depression Test')

    classes_anx = ('0','1','2','3')
    cf_matrix_anx = confusion_matrix(label_anx_list, predicted_list_anx)
    df_cm = pd.DataFrame(cf_matrix_anx/np.sum(cf_matrix_anx)*10,index = [i for i in classes_anx], columns=[i for i in classes_anx])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Anxiety Test')

    fig, axs = plt.subplots(3)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Test Loss')
    axs[0].set(xlabel= 'Epoch', ylabel='Loss')
    axs[1].plot(range(epochs), accuracy_dep)
    axs[1].set_title('Test Depression Accuracy')
    axs[1].set(xlabel= 'Epoch', ylabel='Accuracy')
    axs[2].plot(range(epochs), accuracy_anx)
    axs[2].set_title('Test Anxiety Accuracy')
    axs[2].set(xlabel='Epoch', ylabel='Accuracy')
    plt.show()   

Fold 1
Train Epoch: 0 	Loss: 2.9425	Depression Accuracy: 0.2925	Anxiety Accuracy: 0.3304
Train Epoch: 1 	Loss: 2.9218	Depression Accuracy: 0.3070	Anxiety Accuracy: 0.3358
Train Epoch: 2 	Loss: 2.9094	Depression Accuracy: 0.3132	Anxiety Accuracy: 0.3406
Train Epoch: 3 	Loss: 2.8929	Depression Accuracy: 0.3149	Anxiety Accuracy: 0.3422
Train Epoch: 4 	Loss: 2.8790	Depression Accuracy: 0.3166	Anxiety Accuracy: 0.3439
Train Epoch: 5 	Loss: 2.8672	Depression Accuracy: 0.3181	Anxiety Accuracy: 0.3463
Train Epoch: 6 	Loss: 2.8573	Depression Accuracy: 0.3198	Anxiety Accuracy: 0.3489
Train Epoch: 7 	Loss: 2.8492	Depression Accuracy: 0.3212	Anxiety Accuracy: 0.3512
Train Epoch: 8 	Loss: 2.8424	Depression Accuracy: 0.3224	Anxiety Accuracy: 0.3529
Train Epoch: 9 	Loss: 2.8361	Depression Accuracy: 0.3238	Anxiety Accuracy: 0.3550
Train Epoch: 10 	Loss: 2.8286	Depression Accuracy: 0.3249	Anxiety Accuracy: 0.3567
Train Epoch: 11 	Loss: 2.8214	Depression Accuracy: 0.3259	Anxiety Accuracy: 0.3581
Train E